<a href="https://colab.research.google.com/github/VaibhavGIT5048/DTI/blob/main/DTISEM4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Homomorphic Encryption (Addition) using PyCryptodome
2. Role Based Access Controle (RBAC)
3. Cloud Server Access Controle List
4. Handling a Malicious user attemp

THE PROGRAM SIMULATES
  • Encrypting employee salary and address.
	•	Role-based administrator assigning access roles.
	•	Users requesting access (one authorized, one malicious).
	•	Cloud service provider verifying access.

In [ ]:
!pip install pycryptodome
!pip install bcrypt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 4.5 MB/s eta 0:00:00


Python Code Implemenation

In [66]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import base64
import pandas as pd
import bcrypt
import os
import json
import ipywidgets as widgets
from IPython.display import display

# --- File for storing user data ---
USER_DATA_FILE = "user_data.json"

# --- RSA Key Persistence ---
KEY_FILE = "rsa_key.pem"

if os.path.exists(KEY_FILE):
    with open(KEY_FILE, "rb") as f:
        key = RSA.import_key(f.read())
else:
    key = RSA.generate(2048)
    with open(KEY_FILE, "wb") as f:
        f.write(key.export_key())

public_key = key.publickey()
private_key = key

# --- Function to Encrypt Data ---
def encrypt_data(data):
    cipher = PKCS1_OAEP.new(public_key)
    encrypted_data = []
    for value in data:
        encrypted_chunk = cipher.encrypt(str(value).encode())
        encrypted_data.append(base64.b64encode(encrypted_chunk).decode())
    return encrypted_data

# --- Function to Decrypt Data ---
def decrypt_data(encrypted_data):
    cipher = PKCS1_OAEP.new(private_key)
    decrypted_data = []
    for value in encrypted_data:
        decrypted_chunk = cipher.decrypt(base64.b64decode(value))
        decrypted_data.append(decrypted_chunk.decode())
    return decrypted_data

# --- Sample Employee Data ---
data = {
    'Emp_ID': [101, 102, 103, 104, 105],
    'Emp_Salary': [50000, 60000, 55000, 70000, 65000],
    'Emp_Address': ['Address1', 'Address2', 'Address3', 'Address4', 'Address5']
}
employee_data = pd.DataFrame(data)

# Encrypt Salary & Address
employee_data['Encrypted_Salary'] = encrypt_data(employee_data['Emp_Salary'])
employee_data['Encrypted_Address'] = encrypt_data(employee_data['Emp_Address'])

# --- Role-Based Administrator ---
class RoleBasedAdministrator:
    def __init__(self):
        self.roles = {}
        self.passwords = {}
        self.load_users()

    def load_users(self):
        """ Load users from JSON file and correctly restore passwords as hashed bytes """
        if os.path.exists(USER_DATA_FILE):
            with open(USER_DATA_FILE, "r") as f:
                data = json.load(f)
                self.roles = data.get("roles", {})
                self.passwords = {user: data["passwords"][user].encode() for user in data.get("passwords", {})}

    def save_users(self):
        """ Save users, ensuring hashed passwords remain as bytes """
        data = {
            "roles": self.roles,
            "passwords": {user: self.passwords[user].decode() for user in self.passwords}
        }
        with open(USER_DATA_FILE, "w") as f:
            json.dump(data, f)

    def assign_role(self, user_id, role, password):
        """ Assign a role and properly hash the password """
        hashed_password = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
        self.roles[user_id] = role
        self.passwords[user_id] = hashed_password
        self.save_users()

    def get_role(self, user_id):
        return self.roles.get(user_id, 'No Role Assigned')

    def check_password(self, user_id, password):
        """ Check if the password matches the stored hash """
        stored_hash = self.passwords.get(user_id)
        if stored_hash:
            return bcrypt.checkpw(password.encode(), stored_hash)
        return False

# --- Dynamic Authorization ---
class DynamicAuthorization:
    def __init__(self, admin):
        self.admin = admin

    def check_access(self, user_id, password):
        if self.admin.check_password(user_id, password):
            role = self.admin.get_role(user_id)
            if role == 'Authorized':
                return 'Access Granted'
            return 'Access Denied'
        return 'Access Denied: Incorrect Password'

# --- Cloud Service Provider ---
class CloudServiceProvider:
    def __init__(self, admin, auth):
        self.admin = admin
        self.auth = auth

    def decrypt_data(self, encrypted_data):
        decrypted_salaries = decrypt_data(encrypted_data['Encrypted_Salary'])
        decrypted_addresses = decrypt_data(encrypted_data['Encrypted_Address'])
        return pd.DataFrame({
            'Emp_ID': encrypted_data['Emp_ID'],
            'Decrypted_Salary': decrypted_salaries,
            'Decrypted_Address': decrypted_addresses
        })

# --- Initialize System ---
admin = RoleBasedAdministrator()
auth = DynamicAuthorization(admin)
cloud_provider = CloudServiceProvider(admin, auth)

# --- GUI for Admin Panel ---
def admin_panel():
    print("Admin Panel")
    new_user_id = widgets.Text(placeholder="Enter New User ID")
    new_user_password = widgets.Password(placeholder="Enter Password")
    new_user_role = widgets.Dropdown(options=["Authorized", "Malicious"], value="Authorized")

    def add_new_user(_):
        user_id = new_user_id.value
        role = new_user_role.value
        password = new_user_password.value
        admin.assign_role(user_id, role, password)
        print(f"User {user_id} added with role {role}.")

    add_user_button = widgets.Button(description="Add User")
    add_user_button.on_click(add_new_user)

    malicious_user_id = widgets.Text(placeholder="Enter User ID to Flag")

    def flag_malicious_user(_):
        user_id = malicious_user_id.value
        admin.assign_role(user_id, 'Malicious', admin.passwords.get(user_id, b''))
        print(f"User {user_id} flagged as malicious.")

    flag_malicious_button = widgets.Button(description="Flag Malicious")
    flag_malicious_button.on_click(flag_malicious_user)

    display(new_user_id, new_user_password, new_user_role, add_user_button, malicious_user_id, flag_malicious_button)

# --- User Access Request Panel ---
def user_panel():
    print("User Panel")
    user_id_input = widgets.Text(placeholder="Enter User ID")
    password_input = widgets.Password(placeholder="Enter Password")
    result_output = widgets.Output()

    def request_access(_):
        user_id = user_id_input.value
        password = password_input.value
        access_result = cloud_provider.auth.check_access(user_id, password)
        with result_output:
            result_output.clear_output()
            print(f"Access Result for {user_id}: {access_result}")
            if access_result == 'Access Granted':
                decrypted_data = cloud_provider.decrypt_data(employee_data)
                print(decrypted_data)

    request_button = widgets.Button(description="Request Access")
    request_button.on_click(request_access)

    display(user_id_input, password_input, request_button, result_output)

# --- Main Execution ---
choice = input("Do you want to run the Admin Panel or User Panel? (admin/user): ").strip().lower()
if choice == "admin":
    admin_panel()
elif choice == "user":
    user_panel()
else:
    print("Invalid choice. Please restart and enter 'admin' or 'user'.")


Do you want to run the Admin Panel or User Panel? (admin/user): admin
Admin Panel


Text(value='', placeholder='Enter New User ID')

Password(placeholder='Enter Password')

Dropdown(options=('Authorized', 'Malicious'), value='Authorized')

Button(description='Add User', style=ButtonStyle())

Text(value='', placeholder='Enter User ID to Flag')

Button(description='Flag Malicious', style=ButtonStyle())